In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as py
%matplotlib inline
import wget
import requests
import urllib.request
import urllib.parse
import re

In [17]:
#to get all web page data into xml tags
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
values={
    's':'basic',
    'submit':'search'
}
data=urllib.parse.urlencode(values)
data=data.encode('utf-8')
req=urllib.request.Request(url,data)
resp=urllib.request.urlopen(req)
print(resp)
page=resp.read()


In [18]:
#print(page)
#band_title= re.findall(r'<band_title>(.*?)</band_title>',str(page))
#print(band_title)

In [19]:
#read webpage tables using read_html
#https://medium.com/@ageitgey/quick-tip-the-easiest-way-to-grab-data-out-of-a-web-page-in-python-7153cecfca58
#https://towardsdatascience.com/web-scraping-html-tables-with-python-c9baba21059
from lxml import etree
pos_df = pd.read_html(url,header=0)
#print(pos_df)
#So 2 tables in this page 289*3 and 5*31
#so we want extract only first table
import requests
import lxml.html as lh
import pandas as pd
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')
#Check the length of the first 12 rows
#[len(T) for T in tr_elements[:289]]

In [20]:
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print (i,name)
    col.append((name,[]))

1 Postcode
2 Borough
3 Neighbourhood



In [21]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [22]:
[len(C) for (title,C) in col]

[289, 289, 289]

In [25]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
df.columns=['Postcode','Borough','Neighbourhood']
print(df.head(11))
print(df.shape)



   Postcode           Borough       Neighbourhood
0       M1A      Not assigned      Not assigned\n
1       M2A      Not assigned      Not assigned\n
2       M3A        North York         Parkwoods\n
3       M4A        North York  Victoria Village\n
4       M5A  Downtown Toronto      Harbourfront\n
5       M5A  Downtown Toronto       Regent Park\n
6       M6A        North York  Lawrence Heights\n
7       M6A        North York    Lawrence Manor\n
8       M7A      Queen's Park      Not assigned\n
9       M8A      Not assigned      Not assigned\n
10      M9A         Etobicoke  Islington Avenue\n
(289, 3)


In [27]:
#2nd Filter removing rows which are not assigned to Borough
df=df[df['Borough']!='Not assigned']
print(df.head())
print(df.shape)
#now add neighbourd to values
df.loc[df.Neighbourhood == 'Not assigned\n', "Neighbourhood"] = df.Borough

  Postcode           Borough       Neighbourhood
2      M3A        North York         Parkwoods\n
3      M4A        North York  Victoria Village\n
4      M5A  Downtown Toronto      Harbourfront\n
5      M5A  Downtown Toronto       Regent Park\n
6      M6A        North York  Lawrence Heights\n
(212, 3)


In [29]:
#Now need to find Duplicates in Postcode
df2 = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df2.columns = ['Postcode', 'Borough', 'Neighborhood']
df

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n
5,M5A,Downtown Toronto,Regent Park\n
6,M6A,North York,Lawrence Heights\n
7,M6A,North York,Lawrence Manor\n
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue\n
11,M1B,Scarborough,Rouge\n
12,M1B,Scarborough,Malvern\n


#2nd Question

In [35]:
df2 = pd.read_csv('https://cocl.us/Geospatial_data',header=0)
print(df2.columns)
#changing columns in Df2
df2.columns=['Postcode','Latitude', 'Longitude']
df2_merge = pd.merge(df,df2,on=['Postcode'],how='inner')
df2_merge

Index(['Postal Code', 'Latitude', 'Longitude'], dtype='object')


,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763
5,M6A,North York,Lawrence Manor,43.718518,-79.464763
6,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
7,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
8,M1B,Scarborough,Rouge,43.806686,-79.194353
9,M1B,Scarborough,Malvern,43.806686,-79.194353


In [ ]:
#Method 2 checking which method is faster
#df['Postcode'].value_counts
#Trying another method to download data
import numpy as np
import pandas as pd
import requests

from bs4 import BeautifulSoup


source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source, 'html5lib')

postal_codes_dict = {} # initialize an empty dictionary to save the data in
for table_cell in soup.find_all('td'):
    try:
        postal_code = table_cell.p.b.text # get the postal code
        postal_code_investigate = table_cell.span.text
        neighborhoods_data = table_cell.span.text # get the rest of the data in the cell
        borough = neighborhoods_data.split('(')[0] # get the borough in the cell
        
        # if the cell is not assigned then ignore it
        if neighborhoods_data == 'Not assigned':
            neighborhoods = []
        # else process the data and add it to the dictionary
        else:
            postal_codes_dict[postal_code] = {}
            
            try:
                neighborhoods = neighborhoods_data.split('(')[1]
            
                # remove parantheses from neighborhoods string
                neighborhoods = neighborhoods.replace('(', ' ')
                neighborhoods = neighborhoods.replace(')', ' ')

                neighborhoods_names = neighborhoods.split('/')
                neighborhoods_clean = ', '.join([name.strip() for name in neighborhoods_names])
            except:
                borough = borough.strip('\n')
                neighborhoods_clean = borough

                
 
            # add borough and neighborhood to dictionary
            postal_codes_dict[postal_code]['borough'] = borough
            postal_codes_dict[postal_code]['neighborhoods'] = neighborhoods_clean
    except:
        pass
    
# create an empty dataframe
columns = ['PostalCode', 'Borough', 'Neighborhood']
toronto_data = pd.DataFrame(columns=columns)
toronto_data

# populate dataframe with data from dictionary
for ind, postal_code in enumerate(postal_codes_dict):
    borough = postal_codes_dict[postal_code]['borough']
    neighborhood = postal_codes_dict[postal_code]['neighborhoods']
    toronto_data = toronto_data.append({"PostalCode": postal_code, 
                                        "Borough": borough, 
                                        "Neighborhood": neighborhood},
                                        ignore_index=True)

# print number of rows of dataframe
toronto_data.shape[0]

In [31]:
#Method 3 checking which method is faster
r = requests.get(url)
soup = BeautifulSoup(r.text,'lxml')
tab = soup.find('table')
cols = tab.find_all('td')
lenn = len(cols)
postcode = []
borough = []
neighborhood = []

for i in range(0, lenn, 3):
    postcode.append(cols[i].text.strip())
    borough.append(cols[i+1].text.strip())
    neighborhood.append(cols[i+2].text.strip())
df = pd.DataFrame(data=[postcode, borough, neighborhood]).transpose()
df.columns = ['Postcode', 'Borough', 'Neighborhood']
df.drop(df[df['Borough'] == 'Not assigned'].index, inplace=True)
df.loc[df.Neighborhood == 'Not assigned', "Neighborhood"] = df.Borough
df2 = df.groupby(['Postcode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()
df2.columns = ['Postcode', 'Borough', 'Neighborhood']
df

,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern
